In [ ]:
import numpy as np
import cv2
import os
import random
import pickle
i=1
image_folder='val2017/'
lis=os.listdir(image_folder)
print(len(lis))
data=[]
normdata=[]
for im_name in lis:
    imgn=cv2.imread(image_folder+im_name)
    img1=cv2.resize(imgn,(300,300))
    normdata.append(img1)
    cv2.imwrite('norm/{}.jpg'.format(i),img1)


    x1=np.floor(random.uniform(0, 1)*300)
    y1=np.floor(random.uniform(0, 1)*300)
    x2=np.floor(random.uniform(0, 1)*300*0.25)
    y2=np.floor(random.uniform(0, 1)*300*0.25)

    xmin=int(x1-x2)
    if x1-x2 <0:
        xmin=0

    ymin=int(y1-y2)
    if y1-y2<0:
        ymin=0

    xmax=int(x1+x2)
    if x1+x2>300:
        xmax=300
        
    ymax=int(y1+y2)
    if y1+y2>300:
        ymax=300    

    img1[xmin:xmax,ymin:ymax]=0

    
    cv2.imwrite('pro/{}.jpg'.format(i),img1)
    data.append(img1)
    print(i)
    i+=1



In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D ,BatchNormalization
from tensorflow.keras.models import Model

input_img = Input(shape=(300, 300, 3))  # adapt this if using `channels_first` image data format

x = Conv2D(128, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(1024, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)



x = Conv2D(1024, (3, 3), activation='relu', padding='same')(encoded)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(512, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (5, 5), activation='relu')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (5, 5), activation='relu')(x)
x = BatchNormalization()(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (5, 5), activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (5, 5), activation='relu')(x)
x = Conv2D(32, (3, 3), activation='relu')(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid')(x)

autoencoder = Model(input_img, decoded)
autoencoder.summary()
autoencoder.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
import numpy as np

x_train = np.array(data[0:4800]).astype('float32')/255.
x_test = np.array(data[4800:]).astype('float32')/255.
y_train = np.array(normdata[0:4800]).astype('float32')/255.
y_test = np.array(normdata[4800:]).astype('float32')/255.

In [ ]:
!mkdir tmp

In [ ]:
!mkdir tmp/autoencoder

In [ ]:
!tensorboard --logdir=/tmp/autoencoder

In [ ]:
from keras.callbacks import TensorBoard

autoencoder.fit(x_train, y_train,
                epochs=50,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, y_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

In [ ]:
j=autoencoder.predict(x_test[6].reshape(1,300,300,3))
print(np.floor(j*255.).shape)
cv2.imwrite('frame_out.jpg',np.floor(j*255.).reshape(300,300,3))

In [ ]:
k=x_test[6].reshape(1,300,300,3)
cv2.imwrite('frame_org.jpg',np.floor(k*255.).reshape(300,300,3))